# ***Hate Speech And Target Detection Using Prompting & ChatGPT***

This notebook shows the process of performing hate speech and target detection using prompting and ChatGPT.

### ***Import packages***

Before we begin, let's import all the necessary packages for this notebook:

In [ ]:
import time
import pandas as pd
from random import sample
from typing import Tuple, List
from revChatGPT.ChatGPT import Chatbot

### ***Auxiliary methods***

Next, let's define some auxiliary methods:

In [ ]:
def zero_shot_prompt(
    text: str, is_target: bool = False, add_context: bool = True
) -> str:
  """Method to get zero shot prompt

  Args:
      text (str): Hate text to add to prompt
      is_target (bool, optional): Flag to indicate whether we want target
          detection prompt. Defaults to False.
      add_context (bool, optional): Flag to indicate whether to add context or
          not. Defaults to True.

  Returns:
      str: The prompt.
  """
  # Define the context for hate speech
  context = (
      "Hate speech is abusive or threatening speech or writing that expresses"
      " prejudice on the basis of ethnicity, religion, sexual orientation, or"
      " similar grounds."
  )

  # Add context
  prompt = context if add_context else ""

  # Add target detection prompt or hate detection prompt
  if is_target:
    prompt += (
        "The following text contains hate, towards who is the hate"
        f" directed? \n\nText: '{text}'\nAnswer: "
    )
  else:
    prompt += (
        "Is the following text hate speech?, answer yes or no. \n\nText: "
        f"'{text}'\nAnswer: "
    )
  return prompt


In [ ]:
def few_shot_prompt(
    text: str, examples: List[Tuple[str, str]], is_target: bool = False,
    add_context: bool = True
) -> str:
  """Method to get few shot prompt

  Args:
      text (str): Hate text to add to prompt
      examples (List[Tuple[str, str]]): Examples to add to prompt. Each example
          should contain the text and the label.
      is_target (bool, optional): Flag to indicate whether we want target
          detection prompt. Defaults to False.
      add_context (bool, optional): Flag to indicate whether to add context or
          not. Defaults to True.

  Returns:
      str: The prompt.
  """
  # Define the context for hate speech
  context = (
      "Hate speech is abusive or threatening speech or writing that expresses"
      " prejudice on the basis of ethnicity, religion, sexual orientation, or"
      " similar grounds."
  )

  # Add context
  prompt = context if add_context else ""

  # Add target detection prompt or hate detection prompt
  if is_target:
    prompt += (
        "The following text contains hate, towards who is the hate"
        " directed?"
    )
    for example in examples:
      prompt += (f"\n\nText: '{example[0]}'\nAnswer: {example[1]}")
    prompt += (f"\n\nText: '{text}'\nAnswer: ")
  else:
    prompt += "Is the following text hate speech?, answer yes or no."
    for example in examples:
      prompt += (f"\n\nText: '{example[0]}'\nAnswer: {example[1]}")
    prompt += (f"\n\nText: '{text}'\nAnswer: ")
  return prompt

### ***Read data and API token***

Next, let's read the data and api token:

In [ ]:
# Get api tokens
token = None
with open("token.txt", "r") as f:
  token = f.readline().strip()

# Read data
annotated_df = pd.read_csv("annotated_data.csv")

### ***Zero Shot Hate Detection***

Next, let's perform zero shot hate detection:

In [ ]:
# Define the columns
annotated_df["Hate Detection Zero-Shot Prompt"] = ""
annotated_df["Hate Detection Zero-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object
chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 0

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Try to get a response
  try:
    # Get zero shot prompt
    prompt = zero_shot_prompt(text=item.text, is_target=False, add_context=True)
    # Create ChatGPT conversation and get response
    response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
    # Create new conversation
    chatbot.reset_chat()
    # Reset sleep time
    sleep_time = 5
    # Sleep to not overload api
    time.sleep(sleep_time)
    # Add the columns values
    annotated_df.iloc[loop_index,
                      annotated_df.columns.
                      get_loc("Hate Detection Zero-Shot Prompt")] = prompt
    annotated_df.iloc[
        loop_index,
        annotated_df.columns.
        get_loc("Hate Detection Zero-Shot Response")] = response["message"]
    # Update index
    loop_index += 1
    print(loop_index)
  # Expect all error to get response again
  except Exception as e:
    print(e)
    # Increase sleep time
    sleep_time += 20
    # Sleep to not overload api
    time.sleep(sleep_time)

### ***One Shot Hate Detection***

Next, let's perform one shot hate detection:

In [ ]:
# Define label column
label_col = "hate_label"

# Define the columns
annotated_df["Hate Detection One-Shot Prompt"] = ""
annotated_df["Hate Detection One-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object
chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 0

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Try to get a response
  try:
    # Get indexes of all positive (same) examples
    positive_examples_index = annotated_df[annotated_df[label_col] ==
                                           item[label_col]].index.to_list()
    # Sample an example randomly from all positive examples
    examples_index = sample(
        [i for i in positive_examples_index if i != loop_index], k=1
    )
    # Get examples
    examples = [
        tuple(i) for i in annotated_df.iloc[examples_index].
        loc[:, ["text", label_col]].to_numpy().tolist()
    ]
    # Get one shot prompt
    prompt = few_shot_prompt(
        text=item.text, examples=examples, is_target=False, add_context=True
    )
    # Create ChatGPT conversation and get response
    response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
    # Create new conversation
    chatbot.reset_chat()
    # Reset sleep time
    sleep_time = 5
    # Sleep to not overload api
    time.sleep(sleep_time)
    # Add the columns values
    annotated_df.iloc[
        loop_index,
        annotated_df.columns.get_loc("Hate Detection One-Shot Prompt")] = prompt
    annotated_df.iloc[
        loop_index,
        annotated_df.columns.
        get_loc("Hate Detection One-Shot Response")] = response["message"]
    # Update index
    loop_index += 1
    print(loop_index)
  # Expect all error to get response again
  except Exception as e:
    print(e)
    # Increase sleep time
    sleep_time += 20
    # Sleep to not overload api
    time.sleep(sleep_time)

### ***Few Shot Hate Detection***

Next, let's perform few shot hate detection:

In [ ]:
# Define label column
label_col = "hate_label"

# Define the columns
annotated_df["Hate Detection Few-Shot Prompt"] = ""
annotated_df["Hate Detection Few-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object
chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 28

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Try to get a response
  try:
    # Get indexes of all positive (same) examples
    positive_examples_index = annotated_df[annotated_df[label_col] ==
                                           item[label_col]].index.to_list()
    # Sample an example randomly from all positive examples
    examples_index_positive = sample(
        [i for i in positive_examples_index if i != loop_index], k=2
    )
    # Get indexes of all negative (not the same) examples
    negative_examples_index = annotated_df[
        annotated_df[label_col] != item[label_col]].index.to_list()
    # Sample an example randomly from all positive examples
    examples_index_negative = sample(
        [i for i in negative_examples_index if i != loop_index], k=2
    )
    # Combine example indexes
    examples_index = examples_index_positive + examples_index_negative
    # Get examples
    examples = [
        tuple(i) for i in annotated_df.iloc[examples_index].
        loc[:, ["text", label_col]].to_numpy().tolist()
    ]
    # Get one shot prompt
    prompt = few_shot_prompt(
        text=item.text, examples=examples, is_target=False, add_context=True
    )
    # Create ChatGPT conversation and get response
    response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
    # Create new conversation
    chatbot.reset_chat()
    # Reset sleep time
    sleep_time = 5
    # Sleep to not overload api
    time.sleep(sleep_time)
    # Add the columns values
    annotated_df.iloc[
        loop_index,
        annotated_df.columns.get_loc("Hate Detection Few-Shot Prompt")] = prompt
    annotated_df.iloc[
        loop_index,
        annotated_df.columns.
        get_loc("Hate Detection Few-Shot Response")] = response["message"]
    # Update index
    loop_index += 1
    print(loop_index)
  # Expect all error to get response again
  except Exception as e:
    print(e)
    # Increase sleep time
    sleep_time += 20
    # Sleep to not overload api
    time.sleep(sleep_time)

### ***Zero Shot Hate Target Detection***

Next, let's perform zero shot hate target detection:

In [ ]:
# Define the columns
annotated_df["Hate Target Detection Zero-Shot Prompt"] = ""
annotated_df["Hate Target Detection Zero-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object
chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 0

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Check if item is hate speech
  if item.hate_label == "yes":
    # Try to get a response
    try:
      # Get zero shot prompt
      prompt = zero_shot_prompt(
          text=item.text, is_target=True, add_context=True
      )
      # Create ChatGPT conversation and get response
      response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
      # Create new conversation
      chatbot.reset_chat()
      # Reset sleep time
      sleep_time = 5
      # Sleep to not overload api
      time.sleep(sleep_time)
      # Add the columns values
      annotated_df.iloc[
          loop_index,
          annotated_df.columns.
          get_loc("Hate Target Detection Zero-Shot Prompt")] = prompt
      annotated_df.iloc[loop_index,
                        annotated_df.columns.
                        get_loc("Hate Target Detection Zero-Shot Response")
                       ] = response["message"]
      # Update index
      loop_index += 1
      print(loop_index)
    # Expect all error to get response again
    except Exception as e:
      print(e)
      # Increase sleep time
      sleep_time += 20
      # Sleep to not overload api
      time.sleep(sleep_time)

### ***One Shot Hate Target Detection***

Next, let's perform one shot hate target detection:

In [ ]:
# Define label column
label_col = "hate_target_label"

# Define the columns
annotated_df["Hate Target Detection One-Shot Prompt"] = ""
annotated_df["Hate Target Detection One-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object

chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 0

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Check if item is hate speech
  if item.hate_label == "yes":
    # Try to get a response
    try:
      # Get indexes of all positive (same) examples
      positive_examples_index = annotated_df[annotated_df[label_col] ==
                                             item[label_col]].index.to_list()
      # Sample an example randomly from all positive examples
      examples_index = sample(
          [i for i in positive_examples_index if i != loop_index], k=1
      )
      # Get examples
      examples = [
          tuple(i) for i in annotated_df.iloc[examples_index].
          loc[:, ["text", label_col]].to_numpy().tolist()
      ]
      # Get one shot prompt
      prompt = few_shot_prompt(
          text=item.text, examples=examples, is_target=True, add_context=True
      )
      # Create ChatGPT conversation and get response
      response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
      # Create new conversation
      chatbot.reset_chat()
      # Reset sleep time
      sleep_time = 5
      # Sleep to not overload api
      time.sleep(sleep_time)
      # Add the columns values
      annotated_df.iloc[
          loop_index,
          annotated_df.columns.
          get_loc("Hate Target Detection One-Shot Prompt")] = prompt
      annotated_df.iloc[loop_index,
                        annotated_df.columns.
                        get_loc("Hate Target Detection One-Shot Response")
                       ] = response["message"]
      # Update index
      loop_index += 1
      print(loop_index)
    # Expect all error to get response again
    except Exception as e:
      print(e)
      # Increase sleep time
      sleep_time += 20
      # Sleep to not overload api
      time.sleep(sleep_time)
  else:
    loop_index += 1

### ***Few Shot Hate Target Detection***

Next, let's perform few shot hate target detection:

In [ ]:
# Define label column
label_col = "hate_target_label"

# Define the columns
annotated_df["Hate Detection Few-Shot Prompt"] = ""
annotated_df["Hate Detection Few-Shot Response"] = ""

# Define sleep time which will increase if no response
# in order avoid overloading api
sleep_time = 5

# Define api object
chatbot = Chatbot(
    {"session_token": token}, conversation_id=None, parent_id=None
)

# Define starting index
loop_index = 0

# Loop over each item until we get a response
while loop_index < annotated_df.shape[0]:
  # Get index item
  item = annotated_df.iloc[loop_index]
  # Check if item is hate speech
  if item.hate_label == "yes":
    # Try to get a response
    try:
      # Get indexes of all positive (same) examples
      positive_examples_index = annotated_df[
          (annotated_df[label_col] == item[label_col]) &
          (annotated_df.hate_label == "yes")].index.to_list()
      # Sample an example randomly from all positive examples
      examples_index_positive = sample(
          [i for i in positive_examples_index if i != loop_index], k=2
      )
      # Get indexes of all negative (not the same) examples
      negative_examples_index = annotated_df[
          (annotated_df[label_col] != item[label_col]) &
          (annotated_df.hate_label == "yes")].index.to_list()
      # Sample an example randomly from all positive examples
      examples_index_negative = sample(
          [i for i in negative_examples_index if i != loop_index], k=2
      )
      # Combine example indexes
      examples_index = examples_index_positive + examples_index_negative
      # Get examples
      examples = [
          tuple(i) for i in annotated_df.iloc[examples_index].
          loc[:, ["text", label_col]].to_numpy().tolist()
      ]
      # Get one shot prompt
      prompt = few_shot_prompt(
          text=item.text, examples=examples, is_target=True, add_context=True
      )
      # Create ChatGPT conversation and get response
      response = chatbot.ask(prompt, conversation_id=None, parent_id=None)
      # Create new conversation
      chatbot.reset_chat()
      # Reset sleep time
      sleep_time = 5
      # Sleep to not overload api
      time.sleep(sleep_time)
      
      # Add the columns values
      annotated_df.iloc[loop_index,
                        annotated_df.columns.
                        get_loc("Hate Detection Few-Shot Prompt")] = prompt
      annotated_df.iloc[
          loop_index,
          annotated_df.columns.
          get_loc("Hate Detection Few-Shot Response")] = response["message"]
      # Update index
      loop_index += 1
      print(loop_index)
    # Expect all error to get response again
    except Exception as e:
      print(e)
      # Increase sleep time
      sleep_time += 20
      # Sleep to not overload api
      time.sleep(sleep_time)
  else:
    loop_index += 1

### ***Combine Results To Single File***

Next, let's combine all csv to one csv:

In [ ]:
annotated_df.to_csv("hate_detection_results.csv", index=False)